In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import torch
import re

## Instantiate the Model

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [5]:
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

## Encode and Calculate Sentiment 

In [6]:
tokens = tokenizer.encode('Absolute worst!', return_tensors='pt')

In [7]:
#tokens

tensor([[  101, 35925, 43060,   106,   102]])

In [8]:
#tokenizer.decode(tokens[0])

'[CLS] absolute worst! [SEP]'

In [9]:
result = model(tokens)

In [10]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[ 4.9086,  1.6152, -0.7024, -2.8750, -2.1573]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [12]:
int(torch.argmax(result.logits))+1

1

## Collect Reviews

In [33]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
r.text

'<!DOCTYPE html><html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/\x08no-js\x08/,"js");</script><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta http-equiv="Content-Language" content="en-US" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><link rel="mask-icon" sizes="any" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b2bb2fb0ec9c/assets/img/logos/yelp_burst.svg" content="#FF1A1A"><link rel="shortcut icon" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/dcfe403147fc/assets/img/logos/favicon.ico"><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><script>\n            window.yelp = window.yelp || {};\

In [34]:
soup = BeautifulSoup(r.text, 'html.parser')
soup

<!DOCTYPE html>
<html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/no-js/,"js");</script><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="en-US" http-equiv="Content-Language"/><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/><link content="#FF1A1A" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b2bb2fb0ec9c/assets/img/logos/yelp_burst.svg" rel="mask-icon" sizes="any"/><link href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/dcfe403147fc/assets/img/logos/favicon.ico" rel="shortcut icon"/><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><script>
            window.yelp = window.yelp || {};
      

In [35]:
regex = re.compile('.*comment.*')## as all comments are in teh comment class

In [36]:
results = soup.find_all('p', {'class': regex})
results

[<p class="comment__09f24__D0cxf css-qgunke"><span class="raw__09f24__T4Ezm" lang="en">Hotter than I imagine myself to be and firing up the flaming ring this place packs a firey heat punch with a delicious dolls mexican fusion. Pants free or margarita mash this is the place to explore.</span></p>,
 <p class="comment__09f24__D0cxf css-qgunke"><span class="raw__09f24__T4Ezm" lang="en">The food and service here was really good.  It was more like tapas food than Mexican food!  The drinks were amazing too!</span></p>,
 <p class="comment__09f24__D0cxf css-qgunke"><span class="raw__09f24__T4Ezm" lang="en">Visiting from Texas and decided to give this restaurant a try.  We were pleasantly surprised.  While the margaritas are more like martinis, the food was excellent.  More like a tapas, Mexican fusion.  Great way to try different plates.<br/>The real treat was Chelsea, our waitress.  Took the time to explain the menu and offer suggestions.  Always smiling and very pleasant.  Best service we ha

In [37]:
reviews = [result.text for result in results]
reviews

['Hotter than I imagine myself to be and firing up the flaming ring this place packs a firey heat punch with a delicious dolls mexican fusion. Pants free or margarita mash this is the place to explore.',
 'The food and service here was really good. \xa0It was more like tapas food than Mexican food! \xa0The drinks were amazing too!',
 'Visiting from Texas and decided to give this restaurant a try. \xa0We were pleasantly surprised. \xa0While the margaritas are more like martinis, the food was excellent. \xa0More like a tapas, Mexican fusion. \xa0Great way to try different plates.The real treat was Chelsea, our waitress. \xa0Took the time to explain the menu and offer suggestions. \xa0Always smiling and very pleasant. \xa0Best service we have had in Sydney!',
 'Great atmosphere, attentive service, solid margs, and a Tasty menu. The Brisket Tacos were substantial and delicious. The corn ribs??? \xa0Fawgetaboutit! \xa0Unreal. \xa0Wanted to order another plate.',
 "Don't come here expecting 

In [38]:
resultlist = []
for i in reviews:
    tokens = tokenizer.encode(i, return_tensors='pt')
    result = model(tokens)
    print(i)
    print(f'Review : {int(torch.argmax(result.logits))+1}')
    print('-------------------------------------------------------------------------------------------------------------------')

Hotter than I imagine myself to be and firing up the flaming ring this place packs a firey heat punch with a delicious dolls mexican fusion. Pants free or margarita mash this is the place to explore.
Review : 5
-------------------------------------------------------------------------------------------------------------------
The food and service here was really good.  It was more like tapas food than Mexican food!  The drinks were amazing too!
Review : 5
-------------------------------------------------------------------------------------------------------------------
Visiting from Texas and decided to give this restaurant a try.  We were pleasantly surprised.  While the margaritas are more like martinis, the food was excellent.  More like a tapas, Mexican fusion.  Great way to try different plates.The real treat was Chelsea, our waitress.  Took the time to explain the menu and offer suggestions.  Always smiling and very pleasant.  Best service we have had in Sydney!
Review : 5
-------

In [39]:
df = pd.DataFrame(np.array(reviews), columns=['review'])
df

,review
0,Hotter than I imagine myself to be and firing ...
1,The food and service here was really good. It...
2,Visiting from Texas and decided to give this r...
3,"Great atmosphere, attentive service, solid mar..."
4,Don't come here expecting legit Mexican food b...
5,Out of all the restaurants that I tried in Syd...
6,We came here on a Thursday night @ 5pm and by ...
7,The food is fresh and tasty. The scallop cevi...
8,Have been here twice and have absolutely loved...
9,I was pleasantly surprised at what a great job...


In [40]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [42]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512])) ##as our NLP pipeline is limited to 512 tokens only

In [43]:
df

,review,sentiment
0,Hotter than I imagine myself to be and firing ...,5
1,The food and service here was really good. It...,5
2,Visiting from Texas and decided to give this r...,5
3,"Great atmosphere, attentive service, solid mar...",3
4,Don't come here expecting legit Mexican food b...,3
5,Out of all the restaurants that I tried in Syd...,5
6,We came here on a Thursday night @ 5pm and by ...,4
7,The food is fresh and tasty. The scallop cevi...,4
8,Have been here twice and have absolutely loved...,5
9,I was pleasantly surprised at what a great job...,5
